# Projeto 1 - Ciência dos Dados

Nome: Adney Costa Moura

Nome: Lister Ogusuku Ribeiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [27]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [28]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\adney\OneDrive\Documentos\2° semestre - Insper\Ciência dos Dados\ProjetoCdados_AdneyLister


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [29]:
filename = 'nubank.xlsx'

In [30]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Relevância
0                  @nubank você me ilude tão bem 🤩🤩🤩           0
1  vou ter que fazer parcelamento da parcela, eu ...           0
2  @hs_ramao @nubank aumentaram o meu e eu nem us...           1
3  hoje eu mal acordei e tava pegando ônibus o re...           0
4   🤣🤣🤣 @nubank vamos amigar https://t.co/7k6syhkxf0           0

In [31]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Relevância
0  @ogustamlk gu, obrigado por nos avisar sobre i...           0
1  @nubank nubank libera logo meu cartao de crédi...           1
2  nubank foi o melhor cartão que eu já fiz até h...           1
3                            @nubank quer casar cmg?           0
4         aeeee carai, nubank aceitou meu crédito :3           1

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto foi a startup de serviços financeiros Nubank. Consideramos como relevante as operações que envolvem o limite de crédito, interação com o cliente e reclamações.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [48]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [49]:
# Separando cada row do data frame, fazendo um split das palavras e limpando cada frase. Além disso foi filtrado cada row
# as listas de relevância.
lista_relevante = []
lista_irrelevante = []

for i in range(0,500):
    # Separando os rows do DataFrame
    x = cleanup(train.Treinamento[i]).lower().split()
 
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante.append(x[k])
            k+=1

# Tweets Relevantes

In [50]:
# Guardando as palavras como um pd.Series
serie_relevante = pd.Series(lista_relevante)
serie_relevante.head()

0     @hs_ramao
1       @nubank
2    aumentaram
3             o
4           meu
dtype: object

##### Frequências Absolutas

In [51]:
tabela_relevante_absoluta = serie_relevante.value_counts()
tabela_relevante_absoluta

@nubank    135
o          117
nubank      97
de          95
meu         94
          ... 
roupa        1
mesmos       1
tmbm         1
certeza      1
/´¯`•¸       1
Length: 1091, dtype: int64

##### Frequências Relativas

In [38]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

@nubank    0.038516
o          0.033381
nubank     0.027675
de         0.027104
meu        0.026819
             ...   
roupa      0.000285
mesmos     0.000285
tmbm       0.000285
certeza    0.000285
/´¯`•¸     0.000285
Length: 1091, dtype: float64

# Tweets Irrelevantes

In [39]:
# Guardando as palavras como um pd.Series
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante.head()

0    @nubank
1       você
2         me
3      ilude
4        tão
dtype: object

##### Frequências Absolutas

In [40]:
tabela_irrelevante_absoluta = serie_irrelevante.value_counts()
tabela_irrelevante_absoluta

de                       132
@nubank                  125
o                        116
que                      109
a                        106
                        ... 
https//tco/ddkqjxlv0f      1
@out_pex                   1
seus                       1
aumentarem                 1
crypto                     1
Length: 1780, dtype: int64

##### Frequências Relativas

In [41]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa

de                       0.025414
@nubank                  0.024066
o                        0.022333
que                      0.020986
a                        0.020408
                           ...   
https//tco/ddkqjxlv0f    0.000193
@out_pex                 0.000193
seus                     0.000193
aumentarem               0.000193
crypto                   0.000193
Length: 1780, dtype: float64

# Tweets Totais

In [42]:
total = lista_relevante + lista_irrelevante

In [43]:
total_series = pd.Series(total)

##### Frequências Absolutas

In [44]:
tabela_total = total_series.value_counts()
tabela_total

@nubank    260
o          233
de         227
nubank     191
que        166
          ... 
podiase      1
agencia      1
mó           1
with         1
crypto       1
Length: 2392, dtype: int64

##### Frequências Relativas

In [77]:
tabela_total_relativa = total_series.value_counts(True)
tabela_total_relativa

@nubank    0.029888
o          0.026785
de         0.026095
nubank     0.021957
que        0.019083
             ...   
podiase    0.000115
agencia    0.000115
mó         0.000115
with       0.000115
crypto     0.000115
Length: 2392, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [79]:
lista_test = []
lista_relevante_test = []
lista_irrelevante_test = []

for i in range(0,300):
    x = cleanup(test.Teste[i]).lower().split()
    
    lista_test.append(x)
    
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante_test.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante_test.append(x[k])
            k+=1

In [126]:
contador = 0
Pr = len(lista_relevante)/len(total)
Pi = len(lista_irrelevante)/len(total)

while contador < len(lista_test):
    
    acumulador_relevante = 1
    acumulador_irrelevante = 1

    for h in tweet:

        if h not in lista_relevante:
            tabela_relevante_absoluta[h] = 0

        if h not in lista_irrelevante:
            tabela_irrelevante_absoluta[h] = 0

        acumulador_relevante *= ( tabela_relevante_absoluta[h] + 1 ) / ( len(lista_relevante) + len(total) )
        acumulador_irrelevante *= ( tabela_irrelevante_absoluta[h] + 1 ) / ( len(lista_irrelevante) + len(total) ) 
        print('r:{0}  i:{1}'.format(acumulador_relevante,acumulador_irrelevante))
        
    # Probabilidades
    Prob_d_ser_relevante = Pr*acumulador_relevante
    Prob_d_ser_irrelevante = Pi*acumulador_irrelevante
    
    #print('r:{0}  i:{1}'.format(Prob_d_ser_relevante,Prob_d_ser_irrelevante))
 
    
    if Prob_d_ser_relevante > Prob_d_ser_irrelevante:
        test["Relevância_Classificador"] = 1
        
    else:
        test["Relevância_Classificador"] = 0
        
    contador = contador + 1

r:8.194034742707309e-05  i:7.197869430648528e-05
r:6.714220536469443e-09  i:5.180932434066456e-09
r:5.501655634602952e-13  i:1.8645837594711205e-12
r:9.466959056593084e-16  i:6.039463699431399e-15
r:7.757259141751134e-20  i:2.173563556982437e-18
r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135301924775e-75  i:1.8954315674151596e-69
r:9.662627

r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135301924775e-75  i:1.8954315674151596e-69
r:9.662627502334931e-79  i:5.4572275747935207e-73
r:7.917590545997157e-83  i:3.928041153669849e-77
r:6.228193153193437e-85  i:3.760379136529834e-79
r:5.103403108155881e-89  i:5.41334360689532e-83
r:1.2545238712280928e-92  i:3.8964540465668463e-87
r:5.96217

r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135301924775e-75  i:1.8954315674151596e-69
r:9.662627502334931e-79  i:5.4572275747935207e-73
r:7.917590545997157e-83  i:3.928041153669849e-77
r:6.228193153193437e-85  i:3.760379136529834e-79
r:5.103403108155881e-89  i:5.41334360689532e-83
r:1.2545238712280928e-92  i:3.8964540465668463e-87
r:5.962175068111225e-95  i:3.0850784216681283e-89
r:1.465628089506201e-98  i:1.1102995831239214e-92
r:2.6420696467663406e-101  i:4.235649456961624e-95
r:6.927747353041863e-104  i:7.926789453753849e-98
r:3.40

r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135301924775e-75  i:1.8954315674151596e-69
r:9.662627502334931e-79  i:5.4572275747935207e-73
r:7.917590545997157e-83  i:3.928041153669849e-77
r:6.228193153193437e-85  i:3.760379136529834e-79
r:5.103403108155881e-89  i:5.41334360689532e-83
r:1.2545238712280928e-92  i:3.8964540465668463e-87
r:5.962175068111225e-95  i:3.0850784216681283e-89
r:1.465628089506201e-98  i:1.1102995831239214e-92
r:2.6420696467663406e-101  i:4.235649456961624e-95
r:6.927747353041863e-104  i:7.926789453753849e-98
r:3.405972149971417e-106  i:3.252191743064632e-100
r:2.79086541295593e-110  i:2.3408851530012468e-104
r:5.031058594315836e-113  i:3.032889423020402e-107
r:4.122466891442016e-117  i:2.1830342064495802e-111
r:6.755927386827296e-121  i:7.85659759033175e-115
r:8.194034742707309e-05  i:7.197869430648528e-05
r:6.714220536469443e-09  i:5.180932434066456e-09
r:5.501655634602952e-13  i:1.8645837594711205e-12
r:9.466959056593084e-16  i:6.039463699431399e-15
r:

r:2.6420696467663406e-101  i:4.235649456961624e-95
r:6.927747353041863e-104  i:7.926789453753849e-98
r:3.405972149971417e-106  i:3.252191743064632e-100
r:2.79086541295593e-110  i:2.3408851530012468e-104
r:5.031058594315836e-113  i:3.032889423020402e-107
r:4.122466891442016e-117  i:2.1830342064495802e-111
r:6.755927386827296e-121  i:7.85659759033175e-115
r:8.194034742707309e-05  i:7.197869430648528e-05
r:6.714220536469443e-09  i:5.180932434066456e-09
r:5.501655634602952e-13  i:1.8645837594711205e-12
r:9.466959056593084e-16  i:6.039463699431399e-15
r:7.757259141751134e-20  i:2.173563556982437e-18
r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45


r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135301924775e-75  i:1.8954315674151596e-69
r:9.662627502334931e-79  i:5.4572275747935207e-73
r:7.917590545997157e-83  i:3.928041153669849e-77
r:6.228193153193437e-85  i:3.760379136529834e-79
r:5.103403108155881e-89  i:5.41334360689532e-83
r:1.2545238712280928e-92  i:3.8964540465668463e-87
r:5.96217

r:6.755927386827296e-121  i:7.85659759033175e-115
r:8.194034742707309e-05  i:7.197869430648528e-05
r:6.714220536469443e-09  i:5.180932434066456e-09
r:5.501655634602952e-13  i:1.8645837594711205e-12
r:9.466959056593084e-16  i:6.039463699431399e-15
r:7.757259141751134e-20  i:2.173563556982437e-18
r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135

r:6.755927386827296e-121  i:7.85659759033175e-115
r:8.194034742707309e-05  i:7.197869430648528e-05
r:6.714220536469443e-09  i:5.180932434066456e-09
r:5.501655634602952e-13  i:1.8645837594711205e-12
r:9.466959056593084e-16  i:6.039463699431399e-15
r:7.757259141751134e-20  i:2.173563556982437e-18
r:6.3563250915692675e-24  i:3.129005336475113e-22
r:5.20839486362608e-28  i:1.801773748780026e-25
r:3.4142214773032317e-31  i:3.2422330468221875e-28
r:3.077387434475217e-34  i:2.333717013475986e-31
r:1.2860271972979028e-36  i:1.0750585824693233e-33
r:1.0537751534725523e-40  i:1.5476262613824564e-37
r:1.726934043711164e-44  i:4.45584470274946e-41
r:1.4150557552533302e-48  i:6.414517674727503e-45
r:1.1595016021413718e-52  i:3.693668854662062e-48
r:2.850298923651357e-56  i:7.975963840773185e-52
r:4.6710896815000934e-60  i:1.72229838928378e-55
r:7.655014227302676e-64  i:1.2396878926680918e-59
r:6.272545253443687e-68  i:4.4615557930903757e-63
r:5.13974537319214e-72  i:6.422739211243613e-67
r:5.896135

In [122]:
test.head()

Teste  Relevância  \
0  @ogustamlk gu, obrigado por nos avisar sobre i...           0   
1  @nubank nubank libera logo meu cartao de crédi...           1   
2  nubank foi o melhor cartão que eu já fiz até h...           1   
3                            @nubank quer casar cmg?           0   
4         aeeee carai, nubank aceitou meu crédito :3           1   

   Relevância_Classificador  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0

In [74]:
# Tweet a ser classificado
print(lista_test[0])
tweet = lista_test[0]

['@ogustamlk', 'gu', 'obrigado', 'por', 'nos', 'avisar', 'sobre', 'isso', 'como', 'não', 'podemos', 'passar', 'informações', 'sobre', 'outros', 'clientes', 'poderia', 'nos', 'chamar', 'na', 'dm', 'precisamos', 'de', 'alguns', 'dados', 'que', 'estão', 'no', 'cartão', 'pra', 'agirmos', 'mais', 'rapidamente', '😉']


In [106]:
# Probabilidade (Exercício 1 - Aula 08)
Pr = len(lista_relevante)/len(total)
Pi = len(lista_irrelevante)/len(total)
print("Probabilidade de ser relevante: {0}".format(len(lista_relevante)/len(total)))
print("Probabilidade de ser irrelevante: {0}".format(len(lista_irrelevante)/len(total)))

Probabilidade de ser relevante: 0.4029198758477986
Probabilidade de ser irrelevante: 0.5970801241522014


In [120]:
 # Usando o suavizador de Laplace
acumulador_relevante = 1
acumulador_irrelevante = 1

for h in tweet:
    
    if h not in lista_relevante:
        tabela_relevante_absoluta[h] = 0
        
    if h not in lista_irrelevante:
        tabela_irrelevante_absoluta[h] = 0
        
    acumulador_relevante *= ( tabela_relevante_absoluta[h] + 1 ) / ( len(lista_relevante) + len(total) )
    acumulador_irrelevante *= ( tabela_irrelevante_absoluta[h] + 1 ) / ( len(lista_irrelevante) + len(total) )    
    
print("Acumulador relevante = {0}".format(acumulador_relevante))
print("Acumulador irrelevante = {0}".format(acumulador_irrelevante))

Acumulador relevante = 6.755927386827296e-121
Acumulador irrelevante = 7.85659759033175e-115


In [121]:
# Probabilidade Final
print(Pr*acumulador_relevante)
print(Pi*acumulador_irrelevante)

2.7220974239371965e-121
4.691018264649167e-115


In [ ]:
# Como a segunda probabilidade é maior, o tweet tem uma maior probabilidade de ser irrelevante

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**